Importing the dependencies

In [2]:
import numpy as nmp
import matplotlib.pyplot as pltt

import keras
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing import image

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Loading the dataset

In [4]:
trn_dtgen = image.ImageDataGenerator(
    rescale = 1/255, horizontal_flip = True, zoom_range = 0.2, shear_range = 0.2
)

trn_dt = trn_dtgen.flow_from_directory(directory="/content/drive/MyDrive/Dataset/Train", target_size=(256,256), batch_size=16, 
                                               class_mode = 'binary'
                                               )

Found 288 images belonging to 2 classes.


In [5]:
trn_dt.class_indices

{'Covid': 0, 'Normal': 1}

In [6]:
tst_dtgen = image.ImageDataGenerator(
    rescale = 1/255
)

tst_dt = tst_dtgen.flow_from_directory(directory="/content/drive/MyDrive/Dataset/Val", target_size=(256,256), batch_size=16, 
                                               class_mode = 'binary'
                                               )

Found 60 images belonging to 2 classes.


# CNN Model

In [7]:
modl = Sequential()

modl.add(Conv2D(filters = 32, kernel_size=(3,3), activation='relu', input_shape= (256,256,3)))

modl.add(Conv2D(filters = 64, kernel_size=(3,3), activation='relu'))
modl.add(MaxPool2D())
modl.add(Dropout(rate=0.25))

modl.add(Conv2D(filters = 128, kernel_size=(3,3), activation='relu'))
modl.add(MaxPool2D())
modl.add(Dropout(rate=0.25))

modl.add(Flatten())
modl.add(Dense(units=64, activation='relu'))
modl.add(Dropout(rate=0.50))
modl.add(Dense(units=1, activation='sigmoid'))

modl.compile(loss = keras.losses.binary_crossentropy, optimizer = 'adam', metrics = ['acc'])

In [8]:
modl.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 126, 126, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 128)      0         
 2D)                                                    

In [ ]:
modl.fit_generator(trn_dt, steps_per_epoch=8, epochs=20, validation_steps=2, validation_data=tst_dt)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/20
8/8 [==============================] - 75s 9s/step - loss: 2.6789 - acc: 0.6172 - val_loss: 0.6427 - val_acc: 0.8750
Epoch 2/20
8/8 [==============================] - 59s 7s/step - loss: 0.6071 - acc: 0.6328 - val_loss: 0.6297 - val_acc: 0.9688
Epoch 3/20
8/8 [==============================] - 58s 7s/step - loss: 0.4007 - acc: 0.8594 - val_loss: 0.4365 - val_acc: 0.9062
Epoch 4/20
8/8 [==============================] - 59s 7s/step - loss: 0.3509 - acc: 0.8125 - val_loss: 0.4692 - val_acc: 0.9062
Epoch 5/20
8/8 [==============================] - 57s 7s/step - loss: 0.3280 - acc: 0.8438 - val_loss: 0.1398 - val_acc: 0.9688
Epoch 6/20
8/8 [==============================] - 59s 7s/step - loss: 0.3663 - acc: 0.8594 - val_loss: 0.2933 - val_acc: 0.9062
Epoch 7/20
8/8 [==============================] - 58s 7s/step - loss: 0.2285 - acc: 0.9219 - val_loss: 0.1889 - val_acc: 0.9688
Epoch 8/20
8/8 [==============================] - 60s 8s/step - loss: 0.2621 - acc: 0.9375 - val_loss: 0

In [ ]:
path = "/content/drive/MyDrive/Dataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg"
imag = image.load_imag(path, target_size=(256,256))

imag = image.imag_to_array(imag)/255
img = nmp.array([imag])
imag.shape

(1, 256, 256, 3)

In [ ]:
modl.predict(imag)

array([[0.00307155]], dtype=float32)

In [ ]:
if modl.predict(imag)<0.5:
  print("Covid")
else:
  print("Non-covid")

Covid
